# Example notebook

In [1]:
%load_ext autoreload
%autoreload 2
from multi_condition_comparisions.tl.de import StatsmodelsDE
import scanpy as sc

/home/sturm/apps/micromamba/envs/multi-condition-comparisons/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.datasets.pbmc68k_reduced()
adata.obs

,bulk_labels,n_genes,percent_mito,n_counts,S_score,G2M_score,phase,louvain
index,,,,,,,,
AAAGCCTGGCTAAC-1,CD14+ Monocyte,1003,0.023856,2557.0,-0.119160,-0.816889,G1,1
AAATTCGATGCACA-1,Dendritic,1080,0.027458,2695.0,0.067026,-0.889498,S,1
AACACGTGGTCTTT-1,CD56+ NK,1228,0.016819,3389.0,-0.147977,-0.941749,G1,3
AAGTGCACGTGCTA-1,CD4+/CD25 T Reg,1007,0.011797,2204.0,0.065216,1.469291,G2M,9
ACACGAACGGAGTG-1,Dendritic,1178,0.017277,3878.0,-0.122974,-0.868185,G1,2
...,...,...,...,...,...,...,...,...
TGGCACCTCCAACA-8,Dendritic,1166,0.008840,3733.0,-0.124456,-0.867484,G1,2
TGTGAGTGCTTTAC-8,Dendritic,1014,0.022068,2311.0,-0.298056,-0.649070,G1,1
TGTTACTGGCGATT-8,CD4+/CD25 T Reg,1079,0.012821,3354.0,0.216895,-0.527338,S,0


In [3]:
mod = StatsmodelsDE(adata, "~ phase")

In [4]:
mod.fit()

  0%|          | 0/765 [00:00<?, ?it/s]

100%|██████████| 765/765 [00:00<00:00, 1066.61it/s]


In [16]:
mod._test_single_contrast([1, 0, -1])

100%|██████████| 765/765 [00:00<00:00, 11990.49it/s]


,variable,pvalue,tvalue,sd,fold_change
638,PCNA,6.873154555851182e-22,-9.947205,0.105508,-1.049505
16,STMN1,1.3069700136706127e-17,-8.774957,0.099092,-0.869528
187,HMGB2,1.7649116982121878e-12,-7.182227,0.074164,-0.532659
296,MCM7,1.4949033479878358e-08,-5.729981,0.118656,-0.679896
435,CTSW,2.2184522046497205e-08,-5.659489,0.119954,-0.678879
...,...,...,...,...,...
631,PSMG2,0.9846145981246606,-0.019291,0.112236,-0.002165
331,CD40LG,0.9910297219017024,-0.011247,0.125846,-0.001415
584,PSMD7,0.9920337254875353,-0.009988,0.118428,-0.001183
212,ATP6V0E1,0.9963409614614567,-0.004588,0.114666,-0.000526
